In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('./names.txt').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [8]:
chars = sorted(list(set(''.join(words)))) #lol
token_lookup = {c: i+1 for i, c in enumerate(chars)}
token_lookup['.'] = 0
char_lookup = {i:c for c, i in token_lookup.items()}
char_lookup

{1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z',
 0: '.'}

In [17]:
# build the dataset

BLOCK_SIZE = 3 # context size to give the model in order to predict the next character
X, Y = [], []

for word in words[:3]:
    
    padded_word = f".{word}."
    tokenized_word = [token_lookup[c] for c in padded_word]
    for i in range(len(tokenized_word)-BLOCK_SIZE):
        X.append(tokenized_word[i:i+BLOCK_SIZE])
        Y.append(tokenized_word[i+BLOCK_SIZE])
        
X = torch.tensor(X)
Y = torch.tensor(Y)
X, Y

(tensor([[ 0,  5, 13],
         [ 5, 13, 13],
         [13, 13,  1],
         [ 0, 15, 12],
         [15, 12,  9],
         [12,  9, 22],
         [ 9, 22,  9],
         [22,  9,  1],
         [ 0,  1, 22],
         [ 1, 22,  1]]),
 tensor([13,  1,  0,  9, 22,  9,  1,  0,  1,  0]))